In [1]:
pip install xgboost scikit-learn pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

In [3]:
data = pd.read_csv("orders.csv")

In [10]:
le_payment = LabelEncoder()
data['payment_type'] = le_payment.fit_transform(data['payment_type'])

In [13]:
le_product = LabelEncoder()
data['product_type'] = le_product.fit_transform(data['product_type'])


In [14]:
le_location = LabelEncoder()
data['location_type'] = le_location.fit_transform(data['location_type'])

In [15]:
scaler = StandardScaler()
data[['customer_rto_history']] = scaler.fit_transform(data[['customer_rto_history']])

In [16]:
print(data.dtypes)

order_id                  int64
address_complete          int64
payment_type              int64
customer_rto_history    float64
location_type             int64
product_type              int64
is_rto                    int64
dtype: object


In [17]:
X = data[['address_complete', 'payment_type', 'customer_rto_history', 'location_type', 'product_type']]
y = data['is_rto']

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [19]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [20]:
y_pred_prob = model.predict_proba(X_test)[:, 1]
auc_score = roc_auc_score(y_test, y_pred_prob)

In [21]:

print("RTO Prediction AUC Score:", auc_score)


RTO Prediction AUC Score: 0.5010851591566763


In [24]:
print("Payment classes:", le_payment.classes_)
print("Location classes:", le_location.classes_)
print("Product classes:", le_product.classes_)

Payment classes: [0 1]
Location classes: [0 1]
Product classes: ['Beauty' 'Clothing' 'Electronics' 'Home']


In [26]:
new_orders = pd.DataFrame({
    'address_complete': [1, 0],
    'payment_type': le_payment.transform(['COD', 'Prepaid']),
    'customer_rto_history': scaler.transform([[0], [1]]).ravel(),
    'location_type': le_location.transform(['Urban', 'Rural']),
    'product_type': le_product.transform(['Electronics', 'Clothing'])
})

rto_scores = model.predict_proba(new_orders)[:, 1]
print("Predicted RTO Risk Scores:", rto_scores)



ValueError: invalid literal for int() with base 10: 'COD'